## Подготовка данных

In [128]:
import pandas as pd
# читаем данные из csv
df = pd.read_csv('dataset.csv')
df = df.set_index('variant')
df

,email_inputed,email_not_inputed
variant,,
show,75,38
hide,43,66


In [129]:
# составляем таблицу сопряженности: добавим сумму по рядам и столбцам для вычисления критерия хи-квадрат

df['sum'] = df.email_inputed + df.email_not_inputed
res = df.sum(axis=0).rename('sum')
df = df.append(res)

In [130]:
df

,email_inputed,email_not_inputed,sum
variant,,,
show,75,38,113
hide,43,66,109
sum,118,104,222


Напомним, 

Н<sub>0</sub>: М(х) = М(y), где x и y - количество пользователей, совершивших ввод почты в каждой из групп.

Н<sub>А</sub>: М(х) ≠ М(y)

## Критерий хи-квадрат

In [131]:
# построим таблицу ожидаемых значений
expectations = pd.DataFrame(None, index=df.index[0:2], columns=df.columns[0:2])

for row in range(0, 2):
    for column in range(0, 2):
        row_name = df.index[row]
        column_name = df.columns[column]
        expectations.loc[row_name, column_name] = (df.loc[row_name, 'sum'] * df.loc['sum', column_name]) / df.loc['sum', 'sum']

expectations

,email_inputed,email_not_inputed
variant,,
show,60.0631,52.9369
hide,57.9369,51.0631


In [132]:
def chi_squared(real, expect):
    res = 0
    
    for i in range(0, 4):
        res += ((real[i] - expect[i]) ** 2) / expect[i]
        
    return res

In [133]:
real = df.loc['show'].values[0:2].tolist() + df.loc['hide'].values[0:2].tolist()
expect = expectations.values.flatten().tolist()

print('Значение критерия хи-квадрат равно ', chi_squared(real, expect))

Значение критерия хи-квадрат равно  16.149598518662636


### Критерий хи-квадрат: вывод
Определяем число степеней свободы по формуле: f = (r – 1) × (c – 1). У нас число степеней свободы равно 1.
Сравниваем полученное значение критерия χ<sup>2</sup> с критическими: 16.15 > 3.841 (при α = 0.05), 16.15 > 6.635 (при α = 0.01), следовательно зависимость ввода корпоративной почты от наличия баннера в шапке сайта статистически значима, нулевая гипотеза не принимается.

## t-статистика

Дисперсии неизвестны (но предполагается, что одинаковы), поэтому для проверки равенства мат. ожиданий используем следующую формулу для критерия Стьюдента:

$t=\frac{\bar{X}-\bar{Y}}{\sqrt{(n_{X}-1)\bar{\sigma}_{X}^{2}+(n_{Y}-1)\bar{\sigma}_{Y}^{2}}}\cdot\sqrt{\frac{n_{X}\cdot n_{Y}(n_{X}+n_{Y}-2))}{ n_{X}+n_{Y}}}\sim t(n_{X}+ n_{Y}-2)%0$ , где:

$\bar{X} = \frac{\sum_{i=1}^{n} X_{i}}{n_{X}}%0$

$\bar{\sigma}_{X}^{2} = \frac{\sum_{i=1}^{n} (X{i} - \bar{X})^{2}}{n_X -1}%0$

In [134]:
show_mean = df.loc['show', 'email_inputed'] / df.loc['show', 'sum']
hide_mean = df.loc['hide', 'email_inputed'] / df.loc['hide', 'sum']

temp1 = df.loc['show', 'email_inputed'] * ((1 - show_mean) ** 2) + df.loc['show', 'email_not_inputed'] * ((0 - show_mean) ** 2)
temp2 = df.loc['hide', 'email_inputed'] * ((1 - hide_mean) ** 2) + df.loc['hide', 'email_not_inputed'] * ((0 - hide_mean) ** 2)

In [135]:
import math

t = ((show_mean - hide_mean) / math.sqrt(temp1 + temp2)) * math.sqrt((df.loc['show', 'sum'] * df.loc['hide', 'sum'] * (df.loc['show', 'sum'] + df.loc['hide', 'sum'] - 2)) / (df.loc['show', 'sum'] + df.loc['hide', 'sum']))

print('Значение t-статистики равно ', t)

Значение t-статистики равно  4.154476878958986


### t-статистика: вывод

Количество степеней свободы = 220.
По таблице критических значений критерия Стьюдента находим подходящее значение на уровне значимости 0.001:
t<sub>0.999</sub>=3.3398
Вычисленное значение критерия больше критического, поэтому гипотезу об отсутствии влияния изменения интерфейса на достижение цели не принимаем. 

